# Implement function calling with the Granite-3-8B-Instruct model in Python with WatsonX


**Authors:** Erika Russi, Anna Gutowska, Jess Bozorg

In this tutorial, you will use the IBM® [Granite-3-8B-Instruct model](https://www.ibm.com/granite) now available on watsonx.ai™ to perform custom function calling.  

Traditional [large language models (LLMs)](https://www.ibm.com/topics/large-language-models), like the OpenAI GPT-4 (generative pre-trained transformer) model available through ChatGPT, and the IBM Granite™ models that we'll use in this tutorial, are limited in their knowledge and reasoning. They produce their responses based on the data used to train them and are difficult to adapt to personalized user queries. To obtain the missing information, these [generative AI](https://www.ibm.com/topics/generative-ai) models can integrate external tools within the function calling. This method is one way to avoid fine-tuning a foundation model for each specific use-case. The function calling examples in this tutorial will implement external [API](https://www.ibm.com/topics/api) calls. 

The Granite-3-8B-Instruct model and tokenizer use [natural language processing (NLP)](https://www.ibm.com/topics/natural-language-processing) to parse query syntax. In addition, the models use function descriptions and function parameters to determine the appropriate tool calls. Key information is then extracted from user queries to be passed as function arguments. 

[![Open YouTube video](https://img.youtube.com/vi/cjCYcTPryw8/0.jpg)](https://www.youtube.com/watch?v=cjCYcTPryw8)

This recipe is also available on [IBM.com](https://www.ibm.com/think/tutorials/granite-function-calling).


# Steps

## Step 1. Set up your environment

While you can choose from several tools, this tutorial is best suited for a Jupyter Notebook. Jupyter Notebooks are widely used within data science to combine code with various data sources such as text, images and data visualizations. 

You can run this notebook in [Colab](https://colab.research.google.com/), or download it to your system and [run the notebook locally](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started_with_Jupyter_Locally/Getting_Started_with_Jupyter_Locally.md). 

To avoid Python package dependency conflicts, we recommend setting up a [virtual environment](https://docs.python.org/3/library/venv.html).  

Note, this notebook is compatible with Python 3.11.10 and well as Python 3.10.12, the default in Colab at the time of publishing this tutorial. To check your python version, you can run the `!python --version` command in a code cell.

## Step 2. Set up a Watson Machine Learning service instance and API key

Walk through the [Getting Started with IBM WatsonX](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_WatsonX.ipynb) recipe to ensure you can connect to WatsonX.

## Step 3. Install and import relevant libraries and set up your credentials

We'll need a few libraries and modules for this tutorial. Make sure to import the following ones; if they're not installed, you can resolve this with a quick pip install.

In [ ]:
# installations
%pip install -q git+https://github.com/ibm-granite-community/utils \
    langchain-ibm \
    transformers \
    Jinja2

In [2]:
#imports
import requests
import ast
import re

from transformers import AutoTokenizer

/Users/mtake/.pyenv/versions/3.11.8/envs/granite-snack-cookbook-py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Next, we can prepare our environment by setting the model ID for the `granite-3-8b-instruct` model, and the tokenizer for the same Granite model. 

In [3]:
MODEL_ID = "ibm/granite-3-2-8b-instruct"

TOKENIZER = AutoTokenizer.from_pretrained("ibm-granite/granite-3.2-8b-instruct")

The `get_stock_price` function in this tutorial requires an `AV_STOCK_API_KEY` key. To generate a free `AV_STOCK_API_KEY`, please visit the [Alpha Vantage website](https://www.alphavantage.co/support/#api-key). 

Secondly, the `get_current_weather` function requires a `WEATHER_API_KEY`. To generate one, please [create an account](https://home.openweathermap.org/users/sign_up). Upon creating an account, select the "API Keys" tab to display your free key.

**Store these private keys in a separate `.env` file in the same level of your directory as this notebook.**

In [4]:
from ibm_granite_community.notebook_utils import get_env_var

AV_STOCK_API_KEY = get_env_var("AV_STOCK_API_KEY", "none")

WEATHER_API_KEY = get_env_var("WEATHER_API_KEY", "none")

## Step 4. Define the functions

We can now define our functions. The function's docstring and type information are important for generating the proper tool information.

In this tutorial, the `get_stock_price` function uses the Stock Market Data API available through Alpha Vantage. 

In [5]:
def get_stock_price(ticker: str, date: str) -> dict:
    """
    Retrieves the lowest and highest stock prices for a given ticker and date.

    Args:
        ticker: The stock ticker symbol, e.g., "IBM".
        date: The date in "YYYY-MM-DD" format for which you want to get stock prices.

    Returns:
        A dictionary containing the low and high stock prices on the given date.
    """
    print(f"Getting stock price for {ticker} on {date}")
    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={AV_STOCK_API_KEY}"
        stock_data = requests.get(stock_url)
        stock_low = stock_data.json()["Time Series (Daily)"][date]["3. low"]
        stock_high = stock_data.json()["Time Series (Daily)"][date]["2. high"]
        return {
            "low": stock_low,
            "high": stock_high
        }
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return {
            "low": "none",
            "high": "none"
        }

In [6]:
def get_stock_price_ja(ticker: str, date: str) -> dict:
    """
    指定されたティッカーと日付の最安値と最高値を取得します。

    Args:
        ticker: 株券のティッカーシンボル、例えば 「IBM」。
        date: 株価を取得したい日付を「YYYY-MM-DD」形式で指定します。

    Returns:
        指定された日付の株価の安値と高値を含む辞書。
    """
    print(f"Getting stock price for {ticker} on {date}")
    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={AV_STOCK_API_KEY}"
        stock_data = requests.get(stock_url)
        stock_low = stock_data.json()["Time Series (Daily)"][date]["3. low"]
        stock_high = stock_data.json()["Time Series (Daily)"][date]["2. high"]
        return {
            "low": stock_low,
            "high": stock_high
        }
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return {
            "low": "none",
            "high": "none"
        }

The `get_current_weather` function retrieves the real-time weather in a given location using the Current Weather Data API via [OpenWeather](https://openweathermap.org/api). 

In [7]:
def get_current_weather(location: str) -> dict:
    """
    Fetches the current weather for a given location (default: San Francisco).

    Args:
        location: The name of the city for which to retrieve the weather information.

    Returns:
        A dictionary containing weather information such as temperature, weather description, and humidity.
    """
    print(f"Getting current weather for {location}")

    try:
        # API request to fetch weather data
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
        weather_data = requests.get(weather_url)
        data = weather_data.json()
        # Extracting relevant weather details
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]

        # Returning weather details
        return {
            "description": weather_description,
            "temperature": temperature,
            "humidity": humidity
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {
            "description": "none",
            "temperature": "none",
            "humidity": "none"
        }

In [8]:
def get_current_weather_ja(location: str) -> dict:
    """
    指定した場所（デフォルト：サンフランシスコ）の現在の天気を取得します。

    Args:
        location: 気象情報を取得する都市名。

    Returns:
        気温、天気予報、湿度などの気象情報を含む辞書。
    """
    print(f"Getting current weather for {location}")

    try:
        # API request to fetch weather data
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
        weather_data = requests.get(weather_url)
        data = weather_data.json()
        # Extracting relevant weather details
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]

        # Returning weather details
        return {
            "description": weather_description,
            "temperature": temperature,
            "humidity": humidity
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {
            "description": "none",
            "temperature": "none",
            "humidity": "none"
        }

## Step 5. Set up the API request

Now that our functions are defined, we can create a function that generates a watsonx API request for the provided instructions the watsonx API endpoint. We will use this function each time we make a request.

In [9]:
from langchain_ibm import WatsonxLLM

def make_api_request(instructions: str) -> str:
    model_parameters = {
        "decoding_method": "greedy",
        "max_new_tokens": 200,
        "repetition_penalty": 1.05,
        "stop_sequences": [TOKENIZER.eos_token]
    }
    model = WatsonxLLM(
        model_id=MODEL_ID,
        url= get_env_var("WATSONX_URL"),
        apikey=get_env_var("WATSONX_APIKEY"),
        project_id=get_env_var("WATSONX_PROJECT_ID"),
        params=model_parameters
    )
    response = model.invoke(instructions)
    return response


Next, we can create a list of available functions. Here, we declare our function definitions that require the function names, descriptions, parameters and required properties.

In [10]:
from transformers.utils import get_json_schema

tools = [get_json_schema(tool)["function"] for tool in (get_stock_price, get_current_weather)]
tools

[{'name': 'get_stock_price',
  'description': 'Retrieves the lowest and highest stock prices for a given ticker and date.',
  'parameters': {'type': 'object',
   'properties': {'ticker': {'type': 'string',
     'description': 'The stock ticker symbol, e.g., "IBM".'},
    'date': {'type': 'string',
     'description': 'The date in "YYYY-MM-DD" format for which you want to get stock prices.'}},
   'required': ['ticker', 'date']},
  'return': {'type': 'object',
   'description': 'A dictionary containing the low and high stock prices on the given date.'}},
 {'name': 'get_current_weather',
  'description': 'Fetches the current weather for a given location (default: San Francisco).',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The name of the city for which to retrieve the weather information.'}},
   'required': ['location']},
  'return': {'type': 'object',
   'description': 'A dictionary containing weather information such as temp

In [11]:
from transformers.utils import get_json_schema

tools_ja = [get_json_schema(tool)["function"] for tool in (get_stock_price_ja, get_current_weather_ja)]
tools_ja

[{'name': 'get_stock_price_ja',
  'description': '指定されたティッカーと日付の最安値と最高値を取得します。',
  'parameters': {'type': 'object',
   'properties': {'ticker': {'type': 'string',
     'description': '株券のティッカーシンボル、例えば 「IBM」。'},
    'date': {'type': 'string',
     'description': '株価を取得したい日付を「YYYY-MM-DD」形式で指定します。'}},
   'required': ['ticker', 'date']},
  'return': {'type': 'object', 'description': '指定された日付の株価の安値と高値を含む辞書。'}},
 {'name': 'get_current_weather_ja',
  'description': '指定した場所（デフォルト：サンフランシスコ）の現在の天気を取得します。',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': '気象情報を取得する都市名。'}},
   'required': ['location']},
  'return': {'type': 'object', 'description': '気温、天気予報、湿度などの気象情報を含む辞書。'}}]

## Step 6. Perform function calling


### Step 6a. Calling the get_stock_price function

To prepare for the API requests, we must set our `query` used in the tokenizer chat template.

In [12]:
query = "What were the IBM stock prices on March 7, 2025?"

In [13]:
query_ja = "2025年3月7日のIBMの株価は？"

Applying a chat template is useful for breaking up long strings of texts into one or more messages with corresponding labels. This allows the LLM to process the input in a format that it expects. Because we want our output to be in a string format, we can set the `tokenize` parameter to false. The `add_generation_prompt` can be set to true in order to append the tokens indicating the beginning of an assistant message to the output. This will be useful when generating chat completions with the model.

In [14]:
conversation = [
    {"role": "system","content": "You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required."},
    {"role": "user", "content": query },
]

instruction_1 = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, tokenize=False, add_generation_prompt=True)
instruction_1

'<|start_of_role|>system<|end_of_role|>You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required.<|end_of_text|>\n<|start_of_role|>tools<|end_of_role|>[\n    {\n        "name": "get_stock_price",\n        "description": "Retrieves the lowest and highest stock prices for a given ticker and date.",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "ticker": {\n                    "type": "string",\n                    "description": "The stock ticker symbol, e.g., \\"IBM\\"."\n                },\n                "date": {\n                    "type": "string",\n                    "description": "The date in \\"YYYY-MM-DD\\" format for which you want to get stock prices."\n                }\n            },\n            "required": [\n                "ticker",\n                

In [15]:
conversation_ja = [
    {"role": "system","content": "You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required."},
    {"role": "user", "content": query_ja },
]

instruction_1_ja = TOKENIZER.apply_chat_template(conversation=conversation_ja, tools=tools_ja, tokenize=False, add_generation_prompt=True)
instruction_1_ja

'<|start_of_role|>system<|end_of_role|>You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required.<|end_of_text|>\n<|start_of_role|>tools<|end_of_role|>[\n    {\n        "name": "get_stock_price_ja",\n        "description": "指定されたティッカーと日付の最安値と最高値を取得します。",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "ticker": {\n                    "type": "string",\n                    "description": "株券のティッカーシンボル、例えば 「IBM」。"\n                },\n                "date": {\n                    "type": "string",\n                    "description": "株価を取得したい日付を「YYYY-MM-DD」形式で指定します。"\n                }\n            },\n            "required": [\n                "ticker",\n                "date"\n            ]\n        },\n        "return": {\n            "type": "object",\n            "desc

Now, we can call the `make_api_request` function and pass the instructions we generated.

In [16]:
data_1 = make_api_request(instruction_1)
data_1

'<tool_call>[{"name": "get_stock_price", "arguments": {"ticker": "IBM", "date": "2025-03-07"}}]'

In [17]:
data_1_ja = make_api_request(instruction_1_ja)
data_1_ja

'<tool_call>[{"name": "get_stock_price_ja", "arguments": {"ticker": "IBM", "date": "2025-03-07"}}]'

As you can see by the function name in the JSON object produced by the model, the appropriate `get_stock_price` tool use was selected from the set of functions. To run the function, let's extract relevant information from the output. With the function name and arguments extracted, we can call the function. To call the function using its name as a string, we can use the `globals()` function.

In [18]:
def tool_call(llm_response: str):
    tool_request = ast.literal_eval(re.search("({.+})", llm_response).group(0))
    tool_name = tool_request["name"]
    tool_arguments = tool_request["arguments"]
    tool_response = globals()[tool_name](**tool_arguments)
    return tool_response

Get the response from the requested tool.

In [19]:
tool_response = tool_call(data_1)
tool_response

Getting stock price for IBM on 2025-03-07


{'low': '245.1823', 'high': '261.9600'}

In [20]:
tool_response_ja = tool_call(data_1_ja)
tool_response_ja

Getting stock price for IBM on 2025-03-07


{'low': '245.1823', 'high': '261.9600'}

The function successfully retrieved the requested stock price. To generate a synthesized final response, we can pass another prompt to the Granite model along with the information collected from function calling. 

In [21]:
conversation2 = conversation + [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Display the tool response in natural language." },
    {"role": "tool_response", "content": str(tool_response) },
]

instruction_2 = TOKENIZER.apply_chat_template(conversation=conversation2, tools=tools, tokenize=False, add_generation_prompt=True)
data_2 = make_api_request(instruction_2)
data_2

'On March 7, 2025, the lowest IBM stock price was $245.18 and the highest was $261.96.'

In [22]:
conversation2_ja = conversation_ja + [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Display the tool response in natural language." },
    {"role": "tool_response", "content": str(tool_response_ja) },
]

instruction_2_ja = TOKENIZER.apply_chat_template(conversation=conversation2_ja, tools=tools_ja, tokenize=False, add_generation_prompt=True)
data_2_ja = make_api_request(instruction_2_ja)
data_2_ja

'IBMの株価は2025年3月7日の最安値は245.1823ドル、最高値は261.9600ドルでした。'

### Step 6b. Calling the get_current_weather function

As our next query, let’s inquire about the current weather in San Francisco. We can follow the same steps as in Step 5a by adjusting the query.

In [23]:
query = "What is the current weather in San Francisco?"

In [24]:
query_ja = "サンフランシスコの現在の天気は？"

In [25]:
conversation = [
    {"role": "system","content": "You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required."},
    {"role": "user", "content": query },
]

instruction_1 = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, tokenize=False, add_generation_prompt=True)
instruction_1

'<|start_of_role|>system<|end_of_role|>You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required.<|end_of_text|>\n<|start_of_role|>tools<|end_of_role|>[\n    {\n        "name": "get_stock_price",\n        "description": "Retrieves the lowest and highest stock prices for a given ticker and date.",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "ticker": {\n                    "type": "string",\n                    "description": "The stock ticker symbol, e.g., \\"IBM\\"."\n                },\n                "date": {\n                    "type": "string",\n                    "description": "The date in \\"YYYY-MM-DD\\" format for which you want to get stock prices."\n                }\n            },\n            "required": [\n                "ticker",\n                

In [26]:
conversation_ja = [
    {"role": "system","content": "You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required."},
    {"role": "user", "content": query_ja },
]

instruction_1_ja = TOKENIZER.apply_chat_template(conversation=conversation_ja, tools=tools_ja, tokenize=False, add_generation_prompt=True)
instruction_1_ja

'<|start_of_role|>system<|end_of_role|>You are a helpful assistant with access to the following function calls. Your task is to produce a list of function calls necessary to generate response to the user utterance. Use the following function calls as required.<|end_of_text|>\n<|start_of_role|>tools<|end_of_role|>[\n    {\n        "name": "get_stock_price_ja",\n        "description": "指定されたティッカーと日付の最安値と最高値を取得します。",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "ticker": {\n                    "type": "string",\n                    "description": "株券のティッカーシンボル、例えば 「IBM」。"\n                },\n                "date": {\n                    "type": "string",\n                    "description": "株価を取得したい日付を「YYYY-MM-DD」形式で指定します。"\n                }\n            },\n            "required": [\n                "ticker",\n                "date"\n            ]\n        },\n        "return": {\n            "type": "object",\n            "desc

In [27]:
data_1 = make_api_request(instruction_1)
data_1

'<tool_call>[{"name": "get_current_weather", "arguments": {"location": "San Francisco"}}]'

In [28]:
data_1_ja = make_api_request(instruction_1_ja)
data_1_ja

'<tool_call>[{"name": "get_current_weather_ja", "arguments": {"location": "サンフランシスコ"}}]'

Once again, the model decides the appropriate tool choice, in this case `get_current_weather`, and extracts the location correctly. Now, let's call the function with the argument generated by the model.

In [29]:
tool_response = tool_call(data_1)
tool_response

Getting current weather for San Francisco


{'description': 'broken clouds', 'temperature': 12.29, 'humidity': 83}

In [30]:
tool_response_ja = tool_call(data_1_ja)
tool_response_ja

Getting current weather for サンフランシスコ


{'description': 'broken clouds', 'temperature': 12.29, 'humidity': 83}

The function response correctly describes the current weather in San Francisco. Lastly, let's generate a synthesized final response with the results of this function call. 

In [31]:
conversation2 = conversation + [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Display the tool response in natural language." },
    {"role": "tool_response", "content": str(tool_response) },
]

instruction_2 = TOKENIZER.apply_chat_template(conversation=conversation2, tools=tools, tokenize=False, add_generation_prompt=True)
data_2 = make_api_request(instruction_2)
data_2

'The current weather in San Francisco is broken clouds with a temperature of 12.29 degrees and a humidity level of 83%.'

In [32]:
conversation2_ja = conversation_ja + [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Display the tool response in natural language." },
    {"role": "tool_response", "content": str(tool_response_ja) },
]

instruction_2_ja = TOKENIZER.apply_chat_template(conversation=conversation2_ja, tools=tools_ja, tokenize=False, add_generation_prompt=True)
data_2_ja = make_api_request(instruction_2_ja)
data_2_ja

'現在、サンフランシスコの天気は、部分的に雲がかかっています。気温は12.29度で、湿度は83%です。'

## Summary

In this tutorial, you built custom functions and used the Granite-3-8B-Instruct model to determine which function to call based on  key information from user queries. With this information, you called the function with the arguments as stated in the model response. These function calls produce the expected output. Finally, you called the Granite-3-8B-Instruct model again to synthesize the information returned by the functions. 